In [1]:
# Import PySpark libraries only
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, year, month, dayofmonth, quarter, to_date
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor, LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

print("Using PySpark only - no visualization libraries")

Using PySpark only - no visualization libraries


In [2]:
# Create Spark Session
spark = SparkSession.builder \
    .appName("EV_MultiModel_Comparison") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"Spark version: {spark.version}")
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/26 15:05:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 3.5.1
Spark UI: http://10.233.81.94:4040


## 1. Load and Prepare Data

In [3]:
# Load dataset from HDFS
hdfs_path = "hdfs://localhost:9000/des/data/ev_ice_timeseries_1667_locations_120months_200krows_tweaked_for_events_and_noise.csv"

print("Loading data from HDFS...")
df = spark.read.csv(
    hdfs_path,
    header=True,
    inferSchema=True,
    mode="PERMISSIVE",
    nullValue="NULL",
    nanValue="NaN"
)

print(f"✓ Loaded {df.count():,} rows, {len(df.columns)} columns")

Loading data from HDFS...


✓ Loaded 200,040 rows, 35 columns


In [4]:
# Clean data
print("\nCleaning data...")

# Replace 'None' strings with nulls
for column in df.columns:
    df = df.withColumn(
        column,
        F.when(F.col(column).isin(['None', 'NULL', '', 'NaN']), None).otherwise(F.col(column))
    )

# Remove duplicates
original_count = df.count()
df = df.dropDuplicates()
print(f"Removed {original_count - df.count():,} duplicates")

# Impute missing numeric values with median
numeric_cols = [field.name for field in df.schema.fields 
               if str(field.dataType) in ['IntegerType', 'DoubleType', 'FloatType', 'LongType']]
target_col = 'ev_adoption_probability'

for col_name in numeric_cols:
    if col_name != target_col:
        median_val = df.stat.approxQuantile(col_name, [0.5], 0.01)[0]
        df = df.withColumn(col_name, F.when(F.col(col_name).isNull(), median_val).otherwise(F.col(col_name)))

# Fill categorical with 'Unknown'
categorical_cols = [field.name for field in df.schema.fields if str(field.dataType) == 'StringType']
for col_name in categorical_cols:
    df = df.withColumn(col_name, F.when(F.col(col_name).isNull(), 'Unknown').otherwise(F.col(col_name)))

# Drop rows with null target
df = df.filter(F.col(target_col).isNotNull())

print(f"✓ Cleaned dataset: {df.count():,} rows")


Cleaning data...


25/11/26 15:05:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Removed 0 duplicates


✓ Cleaned dataset: 200,040 rows


In [ ]:
# Feature engineering
print("\nEngineering features...")

if 'date' in df.columns:
    df = df.withColumn('date_parsed', to_date(F.col('date')))
    df = df.withColumn('year', year('date_parsed'))
    df = df.withColumn('month', month('date_parsed'))
    df = df.withColumn('day', dayofmonth('date_parsed'))
    df = df.withColumn('quarter', quarter('date_parsed'))

# Define features
exclude_cols = {target_col, 'date', 'date_parsed'}
all_cols = set(df.columns) - exclude_cols

categorical_features = []
numeric_features = []

for col_name in all_cols:
    dtype = type(df.schema[col_name].dataType).__name__
    if dtype == 'StringType':
        categorical_features.append(col_name)
    elif dtype in ['IntegerType', 'DoubleType', 'FloatType', 'LongType']:
        numeric_features.append(col_name)

print(f"Categorical features: {len(categorical_features)}")
print(f"Numeric features: {len(numeric_features)}")
print(f"Total features: {len(categorical_features) + len(numeric_features)}")


Engineering features...
Categorical features: 0
Numeric features: 0
Total features: 0


## 2. Train-Test Split

In [6]:
# Split data: 80% train, 20% test
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

train_count = train_df.count()
test_count = test_df.count()
total_count = train_count + test_count

print(f"Training set: {train_count:,} rows ({train_count/total_count:.1%})")
print(f"Test set: {test_count:,} rows ({test_count/total_count:.1%})")

Training set: 160,219 rows (80.1%)
Test set: 39,821 rows (19.9%)


## 3. Build Feature Pipeline

In [7]:
# Create preprocessing stages (shared across all models)
indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep") 
           for c in categorical_features]
encoders = [OneHotEncoder(inputCol=f"{c}_idx", outputCol=f"{c}_oh") 
           for c in categorical_features]

assembler_inputs = [f"{c}_oh" for c in categorical_features] + numeric_features
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features", handleInvalid="skip")

print(f"Feature pipeline created with {len(assembler_inputs)} input features")

Feature pipeline created with 0 input features


## 4. Model 1: Random Forest Regressor

In [ ]:
print("="*80)
print("MODEL 1: RANDOM FOREST REGRESSOR")
print("="*80)

# Create model
rf = RandomForestRegressor(
    featuresCol="features",
    labelCol=target_col,
    predictionCol="prediction",
    numTrees=100,
    maxDepth=10,
    seed=42
)

# Build pipeline
rf_pipeline = Pipeline(stages=indexers + encoders + [assembler, rf])

# Train
print("\nTraining Random Forest...")
rf_model = rf_pipeline.fit(train_df)
print("✓ Training complete")

# Predict
rf_predictions = rf_model.transform(test_df)

# Evaluate
evaluator_rmse = RegressionEvaluator(labelCol=target_col, predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol=target_col, predictionCol="prediction", metricName="r2")
evaluator_mae = RegressionEvaluator(labelCol=target_col, predictionCol="prediction", metricName="mae")

rf_rmse = evaluator_rmse.evaluate(rf_predictions)
rf_r2 = evaluator_r2.evaluate(rf_predictions)
rf_mae = evaluator_mae.evaluate(rf_predictions)

print(f"\nRandom Forest Results:")
print(f"  RMSE: {rf_rmse:.6f}")
print(f"  R²:   {rf_r2:.6f}")
print(f"  MAE:  {rf_mae:.6f}")
print("="*80)

## 5. Model 2: Gradient Boosting Regressor

In [ ]:
print("="*80)
print("MODEL 2: GRADIENT BOOSTING REGRESSOR (GBT)")
print("="*80)

# Create model
gbt = GBTRegressor(
    featuresCol="features",
    labelCol=target_col,
    predictionCol="prediction",
    maxIter=100,
    maxDepth=5,
    stepSize=0.1,
    seed=42
)

# Build pipeline
gbt_pipeline = Pipeline(stages=indexers + encoders + [assembler, gbt])

# Train
print("\nTraining Gradient Boosting...")
gbt_model = gbt_pipeline.fit(train_df)
print("✓ Training complete")

# Predict
gbt_predictions = gbt_model.transform(test_df)

# Evaluate
gbt_rmse = evaluator_rmse.evaluate(gbt_predictions)
gbt_r2 = evaluator_r2.evaluate(gbt_predictions)
gbt_mae = evaluator_mae.evaluate(gbt_predictions)

print(f"\nGradient Boosting Results:")
print(f"  RMSE: {gbt_rmse:.6f}")
print(f"  R²:   {gbt_r2:.6f}")
print(f"  MAE:  {gbt_mae:.6f}")
print("="*80)

## 6. Model 3: Linear Regression

In [ ]:
print("="*80)
print("MODEL 3: LINEAR REGRESSION")
print("="*80)

# Create model
lr = LinearRegression(
    featuresCol="features",
    labelCol=target_col,
    predictionCol="prediction",
    maxIter=100,
    regParam=0.01,
    elasticNetParam=0.0
)

# Build pipeline
lr_pipeline = Pipeline(stages=indexers + encoders + [assembler, lr])

# Train
print("\nTraining Linear Regression...")
lr_model = lr_pipeline.fit(train_df)
print("✓ Training complete")

# Predict
lr_predictions = lr_model.transform(test_df)

# Evaluate
lr_rmse = evaluator_rmse.evaluate(lr_predictions)
lr_r2 = evaluator_r2.evaluate(lr_predictions)
lr_mae = evaluator_mae.evaluate(lr_predictions)

print(f"\nLinear Regression Results:")
print(f"  RMSE: {lr_rmse:.6f}")
print(f"  R²:   {lr_r2:.6f}")
print(f"  MAE:  {lr_mae:.6f}")
print("="*80)

## 7. Model Comparison

In [ ]:
# Compare models
print("\n" + "="*80)
print("MODEL COMPARISON RESULTS")
print("="*80)

models = ['Random Forest', 'Gradient Boosting', 'Linear Regression']
rmse_vals = [rf_rmse, gbt_rmse, lr_rmse]
r2_vals = [rf_r2, gbt_r2, lr_r2]
mae_vals = [rf_mae, gbt_mae, lr_mae]

print(f"\n{'Model':<20} {'RMSE':<15} {'R²':<15} {'MAE':<15}")
print("-" * 65)
for i in range(len(models)):
    print(f"{models[i]:<20} {rmse_vals[i]:<15.6f} {r2_vals[i]:<15.6f} {mae_vals[i]:<15.6f}")

print("="*80)

## 8. Select Best Model

In [ ]:
# Determine best model
best_idx = r2_vals.index(max(r2_vals))
best_model_name = models[best_idx]
best_r2 = r2_vals[best_idx]
best_rmse = rmse_vals[best_idx]
best_mae = mae_vals[best_idx]

print("\n" + "="*80)
print("🏆 BEST MODEL SELECTED")
print("="*80)
print(f"\nModel: {best_model_name}")
print(f"R² Score: {best_r2:.6f}")
print(f"RMSE: {best_rmse:.6f}")
print(f"MAE: {best_mae:.6f}")
print("\n" + "="*80)

# Set best model
if best_model_name == 'Random Forest':
    print("\n✓ Random Forest provides the best balance of accuracy and interpretability.")
    best_model = rf_model
    best_predictions = rf_predictions
elif best_model_name == 'Gradient Boosting':
    print("\n✓ Gradient Boosting achieves the highest accuracy.")
    best_model = gbt_model
    best_predictions = gbt_predictions
else:
    print("\n✓ Linear Regression is the simplest model.")
    best_model = lr_model
    best_predictions = lr_predictions

## 9. Prediction Analysis

In [ ]:
# Analyze predictions
print(f"\nSample predictions from {best_model_name}:")
best_predictions.select(target_col, 'prediction').show(20)

# Error analysis
predictions_with_error = best_predictions.withColumn(
    'error', 
    F.col('prediction') - F.col(target_col)
).withColumn(
    'abs_error',
    F.abs(F.col('error'))
)

print(f"\nError statistics:")
predictions_with_error.select('error', 'abs_error').describe().show()

## 10. Save Results

In [ ]:
# Save comparison results
from pyspark.sql import Row

results_data = [
    Row(Model=models[i], RMSE=rmse_vals[i], R2=r2_vals[i], MAE=mae_vals[i])
    for i in range(len(models))
]
results_df = spark.createDataFrame(results_data)

results_df.coalesce(1).write.csv('outputs/model_comparison_results', header=True, mode='overwrite')
print("✓ Model comparison results saved to outputs/model_comparison_results/")

# Save best model predictions
best_predictions.select('location', 'date', target_col, 'prediction') \
    .coalesce(1) \
    .write.csv('outputs/best_model_predictions', header=True, mode='overwrite')
print(f"✓ Best model ({best_model_name}) predictions saved to outputs/best_model_predictions/")

## 11. Summary

In [ ]:
# Final summary
print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"\nDataset: {total_count:,} rows ({train_count:,} train, {test_count:,} test)")
print(f"Features: {len(categorical_features) + len(numeric_features)} ({len(categorical_features)} categorical, {len(numeric_features)} numeric)")
print(f"\nModels trained: {len(models)}")
print(f"Best model: {best_model_name}")
print(f"Best R²: {best_r2:.6f}")
print(f"Best RMSE: {best_rmse:.6f}")
print(f"Best MAE: {best_mae:.6f}")
print("\n✓ Multi-model training and comparison complete!")
print("="*80)

In [ ]:
# Stop Spark session
spark.stop()
print("✓ Spark session stopped")